# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [114]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from pickle import dump
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [59]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM MessageTable", engine)
X = df['message'].values
y = df.iloc[:, 4:].values

### 2. Write a tokenization function to process your text data

In [60]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [118]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [84]:
y_pred = pipeline.predict(X_test)
print("Labels:", np.unique(y_pred))
print("Accuracy:", (y_pred == y_test).mean())

Labels: [ 0.  1.]
Accuracy: 0.923901530682


In [87]:
print("Labels:", np.unique(y_pred))
print("Accuracy:", (y_pred == y_test).mean())
y_cols = df.iloc[:, 4:].columns
for c in range(y_pred.shape[1]):
    print("Category:", y_cols[c])
    print("Accuracy:", (y_pred[:, c] == y_test[:, c]).mean())
    print(classification_report(y_test[:, c], y_pred[:, c], labels=labels))

Category: related
Accuracy: 0.734009840098
             precision    recall  f1-score   support

        0.0       0.32      0.11      0.17      1531
        1.0       0.77      0.92      0.84      4973

avg / total       0.67      0.73      0.68      6504

Category: request
Accuracy: 0.819034440344
             precision    recall  f1-score   support

        0.0       0.83      0.98      0.90      5338
        1.0       0.47      0.07      0.12      1166

avg / total       0.76      0.82      0.76      6504

Category: offer
Accuracy: 0.9950799508
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6472
        1.0       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6504

Category: aid_related
Accuracy: 0.554889298893
             precision    recall  f1-score   support

        0.0       0.58      0.82      0.68      3759
        1.0       0.44      0.19      0.27      2745

avg / total       0.52

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [93]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3],
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, n_jobs=-1, verbose=2)

In [94]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total=  59.4s
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total=  58.1s
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total=  58.2s
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), total= 2.0min
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), total= 1.9min
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), total= 1.9min
[CV] clf__estimator__min_samples_split=3, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=3, vect__ngram_range=(1, 1), total=  52.7s
[CV] clf__estimator__min_samples_split=3, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=3, vect__ngram_range=(1, 1), total= 

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 18.5min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [95]:
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'vect__ngram_range': (1, 2)}

In [100]:
cv.best_estimator_.named_steps

{'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 2), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=<function tokenize at 0x7f87d901a2f0>, vocabulary=None),
 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
            n_jobs=

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [101]:
y_pred2 = cv.predict(X_test)
print("Labels:", np.unique(y_pred))
print("Accuracy:", (y_pred2 == y_test).mean())

Labels: [ 0.  1.]
Accuracy: 0.924576329097


In [102]:
y_cols = df.iloc[:, 4:].columns
for c in range(y_pred2.shape[1]):
    print("Category:", y_cols[c])
    print("Accuracy:", (y_pred2[:, c] == y_test[:, c]).mean())
    print(classification_report(y_test[:, c], y_pred2[:, c], labels=labels))

Category: related
Accuracy: 0.741543665437
             precision    recall  f1-score   support

        0.0       0.35      0.11      0.17      1531
        1.0       0.77      0.93      0.85      4973

avg / total       0.67      0.74      0.69      6504

Category: request
Accuracy: 0.818265682657
             precision    recall  f1-score   support

        0.0       0.83      0.98      0.90      5338
        1.0       0.45      0.06      0.10      1166

avg / total       0.76      0.82      0.76      6504

Category: offer
Accuracy: 0.9950799508
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6472
        1.0       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6504

Category: aid_related
Accuracy: 0.555043050431
             precision    recall  f1-score   support

        0.0       0.58      0.83      0.68      3759
        1.0       0.43      0.18      0.25      2745

avg / total       0.52

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [112]:
pipeline2.named_steps

{'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=<function tokenize at 0x7f87d901a2f0>, vocabulary=None),
 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
 'clf': MultiOutputClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
        eta0=0.0, fit_intercept=True, l1_ratio=0.15,
        learning_rate='optimal', loss='hinge', max_iter=1000, n_iter=None,
        n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
        shuffle=True, tol=0.001, verbose=0, warm_start=False),
            n_jobs=1)}

In [116]:
pipeline2 = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(MultinomialNB())),
                    ])
pipeline2.fit(X_train, y_train)
y_pred3 = pipeline2.predict(X_test)
print("Labels:", np.unique(y_pred3))
print("Accuracy:", (y_pred3 == y_test).mean())
y_cols = df.iloc[:, 4:].columns
for c in range(y_pred3.shape[1]):
    print("Category:", y_cols[c])
    print("Accuracy:", (y_pred3[:, c] == y_test[:, c]).mean())
    print(classification_report(y_test[:, c], y_pred3[:, c], labels=labels))

/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Labels: [ 0.  1.]
Accuracy: 0.926442701927
Category: related
Accuracy: 0.764145141451
             precision    recall  f1-score   support

        0.0       0.29      0.00      0.00      1531
        1.0       0.76      1.00      0.87      4973

avg / total       0.65      0.76      0.66      6504

Category: request
Accuracy: 0.820725707257
             precision    recall  f1-score   support

        0.0       0.82      1.00      0.90      5338
        1.0       0.50      0.01      0.01      1166

avg / total       0.76      0.82      0.74      6504

Category: offer
Accuracy: 0.9950799508
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6472
        1.0       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6504

Category: aid_related
Accuracy: 0.57241697417
             precision    recall  f1-score   support

        0.0       0.58      0.95      0.72      3759
        1.0       0.44      0.05  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
model_name = 'model_pipe' + '.pkl'
dump(pipeline, open(model_name, 'wb'))        

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.